In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [4]:
import polars as pl
import duckdb as dd

## Conditions
### spinal_canal_stenosis
### neural_foraminal_narrowing
#### left/right
### subarticular_stenosis
#### left/right

## Levels of vertebrae
#### l1_l2
#### l2_l3
#### l3_l4
#### l4_l5
#### l5_s1

> Each Study has series of images for these three conditions. The series description tells us what type of image is in the dcm file

In [12]:
train_meta_data = pl.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv', low_memory=True)
train_meta_data.head(10)

study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,left_neural_foraminal_narrowing_l5_s1,right_neural_foraminal_narrowing_l1_l2,right_neural_foraminal_narrowing_l2_l3,right_neural_foraminal_narrowing_l3_l4,right_neural_foraminal_narrowing_l4_l5,right_neural_foraminal_narrowing_l5_s1,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
4003253,"""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Moderate""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild"""
4646740,"""Normal/Mild""","""Normal/Mild""","""Moderate""","""Severe""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Moderate""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Moderate""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Severe""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Moderate""","""Moderate""","""Normal/Mild"""
7143189,"""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild"""
8785691,"""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Moderate""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild"""
10728036,"""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Normal/Mild"""
11340341,"""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Moderate""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Moderate""","""Moderate""","""Normal/Mild"""
11943292,"""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild""","""Normal/Mild

In [10]:
train_series_desc = pl.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv', low_memory=True)
train_series_desc.head(10)

study_id,series_id,series_description
i64,i64,str
4003253,702807833,"""Sagittal T2/STIR"""
4003253,1054713880,"""Sagittal T1"""
4003253,2448190387,"""Axial T2"""
4646740,3201256954,"""Axial T2"""
4646740,3486248476,"""Sagittal T1"""
4646740,3666319702,"""Sagittal T2/STIR"""
7143189,132939515,"""Sagittal T2/STIR"""
7143189,1951927562,"""Axial T2"""
7143189,3219733239,"""Sagittal T1"""


In [16]:
dd.sql("select series_description, count(distinct(study_id)) as studies, count(distinct(series_id)) as series \
from train_series_desc group by series_description").pl()

series_description,studies,series
str,i64,i64
"""Sagittal T2/STIR""",1974,1974
"""Sagittal T1""",1973,1980
"""Axial T2""",1975,2340


In [13]:
train_series_desc.filter(pl.col('study_id')==100206310)

study_id,series_id,series_description
i64,i64,str
100206310,1012284084,"""Axial T2"""
100206310,1792451510,"""Sagittal T2/STIR"""
100206310,2092806862,"""Sagittal T1"""


In [ ]:
train_meta_data

In [7]:
pl.Config(fmt_str_lengths=100)
submission = pl.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv', low_memory=True)
## row_id --> study_id _ condition _ level
submission.head(10)

row_id,normal_mild,moderate,severe
str,f64,f64,f64
"""44036939_left_neural_foraminal_narrowing_l1_l2""",0.333333,0.333333,0.333333
"""44036939_left_neural_foraminal_narrowing_l2_l3""",0.333333,0.333333,0.333333
"""44036939_left_neural_foraminal_narrowing_l3_l4""",0.333333,0.333333,0.333333
"""44036939_left_neural_foraminal_narrowing_l4_l5""",0.333333,0.333333,0.333333
"""44036939_left_neural_foraminal_narrowing_l5_s1""",0.333333,0.333333,0.333333
"""44036939_left_subarticular_stenosis_l1_l2""",0.333333,0.333333,0.333333
"""44036939_left_subarticular_stenosis_l2_l3""",0.333333,0.333333,0.333333
"""44036939_left_subarticular_stenosis_l3_l4""",0.333333,0.333333,0.333333
"""44036939_left_subarticular_stenosis_l4_l5""",0.333333,0.333333,0.333333
